# Experiments with ObsLocTAP for ZTF's Rubin SV shadowing

## 1. Introduction
This notebook demonstrates how to look at the ObsLocTAP results that are being published by the [ZTF schedule reporting service](https://zwickytransientfacility.github.io/schedule_reporting_service/) for ZTF's Rubin SV shadowing observations.

### Import needed packages

In [ ]:
from datetime import datetime, timedelta

import numpy as np
from astropy.time import Time
from astropy.table import QTable
from astropy import units as u
from sregion import SRegion
import skyproj
from matplotlib import pyplot as plt

%matplotlib widget

Define observation date, convert to string in the right format and fetch the VOTable of scheduled observations which are in [ObsLocTAP](https://www.ivoa.net/documents/ObsLocTAP/index.html) format. We then convert the three time-related columns to AstroPy `Time` mixin columns.

In [ ]:
obs_date = datetime(2025, 8, 11)
date_str = obs_date.strftime('%Y-%m-%d')
print(f'Searching for observations on {date_str}')

In [ ]:
url = f'http://schedule.ztf.uw.edu/ZTF_ObsLoc_{date_str}.xml'
ztf_obs = QTable.read(url)

In [ ]:
for column in ['t_planning', 't_min', 't_max']:
    t = Time(ztf_obs[column], format='mjd')
    ztf_obs[column] = t

In [ ]:
ztf_obs

### Footprints and regions
The footprint information is in the `s_region` column and is in the unformalized but widely used subset of [STC-S](https://www.ivoa.net/documents/Notes/STC-S/) as detailed in Section 6 of the [TAP 1.0](https://www.ivoa.net/documents/TAP/20100327/REC-TAP-1.0.html) specification. We can use the `sregion` package (need >=1.5 for the `BOX` support TL added) to turn these into `SRegion` objects which can then be converted into `matplotlib.patch` patches or `shapely` polygons

In [ ]:
row = ztf_obs[0]
print(row['s_region'])

In [ ]:
sr = SRegion(row['s_region'])
print(sr.centroid)
print(sr.sky_area(u.deg**2)[0])

### Plotting
We use `skyproj` to make a Mollweide projection plot and label it with the Milky Way band and the ecliptic

In [ ]:
plt.close()
fig, ax = plt.subplots(1, 1, layout='constrained', dpi=150)
sp = skyproj.MollweideSkyproj(ax=ax)
# Draw Milky Way (default is black lines, 1.5 linewidth and +/- 10 degrees of the equator
linewidth = 1.5
color = 'black'
sp.draw_milky_way(label='Milky Way', linewidth=linewidth, color=color)
mw_line = sp.ax.lines[-3]
# Draw ecliptic and label
elon = np.linspace(0, 360, 500)
elat = np.zeros_like(elon)
ec = SkyCoord(lon=elon * u.degree, lat=elat * u.degree, distance=1 * u.au, frame='heliocentricmeanecliptic')
radec = ec.icrs
lon = radec.ra.degree
lat = radec.dec.degree
sp.ax.plot(lon, lat, linewidth=1.0, color='green', linestyle='--', label='Ecliptic')
ecl_line = sp.ax.lines[-1]
legend_handles = [mw_line, ecl_line]
sp.ax.legend(handles=legend_handles, loc='upper right', fontsize='x-small')
ax.grid()

### Turn all the rows (ZTF observed fields) into `SRegion`s and plot them

In [ ]:
for region in ztf_obs['s_region']:
    sr = SRegion(region)
    # Transpose the array of (x,y) points to get x and y arrays
    poly_points = sr.xy[0].T
    sp.draw_polygon(
        poly_points[0], poly_points[1], edgecolor='black', alpha=0.5, facecolor='red', linewidth=0.75, linestyle='--'
    )
night_patches = sp.ax.patches[-1]
night_patches.set_label(date_str)
legend_handles.append(night_patches)

In [ ]:
# Update legend and add title
sp.ax.legend(handles=legend_handles, loc='upper right', fontsize='x-small')
sp.ax.set_title(f'ZTF fields for {date_str}')